In [1]:
import torch
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data

import pandas as pd
import numpy as np
import warnings
import time

warnings.filterwarnings("ignore")

In [2]:
# ===================Train Data======================= #
#
#  输入30个采样点，预测30个采样点。由于每个
#  采样点都包含3个坐标(x,y,z)，所以需要输入
#  维度为90，输出维度为90。
#
#  每段轨迹长度3秒，共90个采样点。最后30个
#  采样点不需要作为输入，前30个采样点不需要
#  作为预测的真实值输出。前60个采样点作为输
#  入
#  
#  N = landmarks.shape[0] - landmarks.shape[0]/9
#
#  range取值是轨迹的条数
# =====================================================

# FC-net architecture
D_in, H_in, H_out, D_out = 30, 10, 10, 15 
Z_dim, Learning_rate = 3, 1e-3
#D_in, H_in, H_out, D_out = 20, 15, 15, 10

landmarks_frame = pd.read_csv('Try.csv')
landmarks = landmarks_frame.as_matrix().astype('float')

In [4]:
plus = np.random.randint(low=1, high=2, size=landmarks.shape)
plus = plus/10000
plus = landmarks + plus
tempo = np.append(landmarks, plus, axis=0)

sub = np.random.randint(low=1, high=4, size=landmarks.shape)
sub = sub/10000
sub = landmarks - sub
landmarks = np.append(tempo, sub, axis=0)
print(landmarks.shape)

(55080, 3)


In [3]:
# Normalization
#maxdata = np.array([1.5, 0.4, 1.2])
#mindata = np.array([0.8, -0.5, 0.6])
maxdata = np.max(landmarks, axis=0)
mindata = np.min(landmarks, axis=0)
landmarks = (landmarks - mindata)/(maxdata - mindata)
landmarks = landmarks.astype('float').reshape(-1, D_in)


N = landmarks.shape[0] * 8 / 9
N = int(N)
count = landmarks.shape[0] / 9
count = int(count)
#print('Landmarks shape: {}'.format(landmarks.shape))
x = torch.zeros(N, D_in)
y = torch.zeros(N, D_out)

for i in range(count):
    x[i*8:i*8+8] = Variable(torch.from_numpy(landmarks[i*9:i*9+8]), requires_grad = False)
    y[i*8:i*8+8] = Variable(torch.from_numpy(landmarks[i*9+1:i*9+9, :D_out]), requires_grad = False)
print(x.shape)

torch.manual_seed(1)    # reproducible
torch_dataset = Data.TensorDataset(x, y)
# 把 dataset 放入 DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,      # torch TensorDataset format
    batch_size=8,               # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)


# ===================Validation Data======================= #
landmarks_frame_val = pd.read_csv('Val.csv')
landmarks_val = landmarks_frame_val.as_matrix().astype('float')

landmarks_val = (landmarks_val - mindata)/(maxdata - mindata)
landmarks_val = landmarks_val.astype('float').reshape(-1, D_in)

#print('Landmarks shape: {}'.format(landmarks_val.shape))

N_val = landmarks_val.shape[0] * 8 / 9
N_val = int(N_val)
count_val = landmarks_val.shape[0] / 9
count_val = int(count_val)

x_val = torch.zeros(N_val, D_in)
y_val = torch.zeros(N_val, D_out)

for i in range(count_val):
    x_val[i*8:i*8+8] = Variable(torch.from_numpy(landmarks_val[i*9:i*9+8]), requires_grad = False)
    y_val[i*8:i*8+8] = Variable(torch.from_numpy(landmarks_val[i*9+1:i*9+9, :D_out]), requires_grad = False)

print(x_val.shape)

torch.Size([376, 30])
torch.Size([120, 30])


In [7]:
# 全连接

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H_in),
    #torch.nn.ReLU(),
    #torch.nn.Linear(H_in, H_out),
    torch.nn.ReLU(),
    torch.nn.Linear(H_out, D_out),
    
)

loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=Learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum=0.9)
#scheduler = ReduceLROnPlateau(optimizer, 'min', 1e-5, 2, threshold=1e-7 )

# 载入训练号的参数
#model.load_state_dict(torch.load('params_prone.pkl'))

start = time.time()
for t in range(2000):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(loss)
    
    # Print and plot every now and then
    if t % 100 == 0:
        print('Iter-{}; Loss: {:.4}'.format(t, loss.data[0]))
end = time.time()
print('FC net cal_time is {}'.format(end-start))

y_pred_val = model(x_val)    
loss_val = loss_fn(y_pred_val, y_val)
print('FC net val_loss is {}'.format(loss_val.item()))

Iter-0; Loss: 2.021e+03
Iter-100; Loss: 220.4
Iter-200; Loss: 131.9
Iter-300; Loss: 79.15
Iter-400; Loss: 51.01
Iter-500; Loss: 36.26
Iter-600; Loss: 27.76
Iter-700; Loss: 22.47
Iter-800; Loss: 19.07
Iter-900; Loss: 16.74
Iter-1000; Loss: 15.02
Iter-1100; Loss: 13.62
Iter-1200; Loss: 12.42
Iter-1300; Loss: 11.39
Iter-1400; Loss: 10.49
Iter-1500; Loss: 9.705
Iter-1600; Loss: 9.019
Iter-1700; Loss: 8.417
Iter-1800; Loss: 7.885
Iter-1900; Loss: 7.415
FC net cal_time is 0.8267889022827148
FC net val_loss is 2.627199649810791


In [8]:
for t in range(18000):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(loss)
    
    # Print and plot every now and then
    if t % 1000 == 0:
        print('Iter-{}; Loss: {:.4}'.format(t, loss.data[0]))

y_pred_val = model(x_val)    
loss_val = loss_fn(y_pred_val, y_val)
print('FC net val_loss is {}'.format(loss_val.item()))

Iter-0; Loss: 6.997
Iter-1000; Loss: 4.36
Iter-2000; Loss: 2.811
Iter-3000; Loss: 2.036
Iter-4000; Loss: 1.57
Iter-5000; Loss: 1.303
Iter-6000; Loss: 1.178
Iter-7000; Loss: 1.102
Iter-8000; Loss: 1.05
Iter-9000; Loss: 1.009
Iter-10000; Loss: 0.967
Iter-11000; Loss: 0.871
Iter-12000; Loss: 0.8308
Iter-13000; Loss: 0.8104
Iter-14000; Loss: 0.7931
Iter-15000; Loss: 0.7803
Iter-16000; Loss: 0.7797
Iter-17000; Loss: 0.7602
FC net val_loss is 0.6690676212310791


In [7]:
""" LSTM

将整个输入看作一个序列，序列长度即维度0的大小
(seq_len, batch_size, dim)
"""
class lstm(torch.nn.Module):
    def __init__(self, d_in, d_out, num_layer, h_in):
        super(lstm, self).__init__()
        self.lstm = torch.nn.LSTM(h_in, h_in, num_layer)
        self.linear1 = torch.nn.Linear(d_in, h_in)
        self.linear2 = torch.nn.Linear(h_in, d_out)
        
        self.num_layer = num_layer
        self.h_in = h_in
        self.hidden = self.init_hidden()
    
    def forward(self, x):
        tem = self.linear1(x)
        out, _ = self.lstm(tem.view(len(tem), 1, -1), self.hidden)
        y_pred = self.linear2(out.view(len(x), -1))
        return y_pred
    
    def init_hidden(self):
        return (torch.zeros(self.num_layer * 1, 1, self.h_in),
               torch.zeros(self.num_layer * 1, 1, self.h_in))


model = lstm(D_in, D_out, 1, H_in)    
loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=Learning_rate)

start = time.time()
for t in range(2000):
    #scheduler.step
    model.hidden = model.init_hidden()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    # Print and plot every now and then
    if t % 100 == 0:
        print('Iter-{}; Loss: {:.4}'.format(t, loss.data[0]))
end = time.time()
print('LSTM cal_time is {}'.format(end-start))

y_pred_val = model(x_val)
y_pred_val = y_pred_val.reshape(y_pred_val.shape[0], D_out)
loss_val = loss_fn(y_pred_val, y_val)
print('LSTM val_loss is {}'.format(loss_val.item()))

Iter-0; Loss: 1.398e+03
Iter-100; Loss: 266.1
Iter-200; Loss: 138.2
Iter-300; Loss: 53.91
Iter-400; Loss: 34.24
Iter-500; Loss: 24.07
Iter-600; Loss: 17.78
Iter-700; Loss: 14.16
Iter-800; Loss: 11.83
Iter-900; Loss: 10.16
Iter-1000; Loss: 8.851
Iter-1100; Loss: 7.774
Iter-1200; Loss: 6.869
Iter-1300; Loss: 6.106
Iter-1400; Loss: 5.461
Iter-1500; Loss: 4.914
Iter-1600; Loss: 4.448
Iter-1700; Loss: 4.046
Iter-1800; Loss: 3.697
Iter-1900; Loss: 3.392
LSTM cal_time is 185.18163895606995
LSTM val_loss is 1.6534544229507446


In [6]:
for t in range(18000):
    #scheduler.step
    model.hidden = model.init_hidden()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    # Print and plot every now and then
    if t % 1000 == 0:
        print('Iter-{}; Loss: {:.4}'.format(t, loss.data[0]))

y_pred_val = model(x_val)
y_pred_val = y_pred_val.reshape(y_pred_val.shape[0], D_out)
loss_val = loss_fn(y_pred_val, y_val)
print('LSTM val_loss is {}'.format(loss_val.item()))

Iter-0; Loss: 3.581
Iter-1000; Loss: 2.054
Iter-2000; Loss: 1.371
Iter-3000; Loss: 1.052
Iter-4000; Loss: 0.8661


KeyboardInterrupt: 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [4]:
""" VAE模型

自定义初始化
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

或调用官方方法
>>> w = torch.empty(3, 5)
>>> torch.nn.init.xavier_normal_(w)
"""

class vae(torch.nn.Module):
    def __init__(self, z_dim, x_dim, y_dim, h_dim):
        super(vae, self).__init__()
        self.Wxh = Variable(torch.nn.init.xavier_normal_(torch.empty(h_dim, x_dim)), requires_grad=True)
        self.bxh = Variable(torch.zeros(h_dim), requires_grad=True)
     
        self.Whz_mu = Variable(torch.nn.init.xavier_normal_(torch.empty(z_dim, h_dim)), requires_grad=True)  # 需要更改
        self.bhz_mu = Variable(torch.zeros(z_dim), requires_grad=True)

        self.Whz_var = Variable(torch.nn.init.xavier_normal_(torch.empty(z_dim, h_dim)), requires_grad=True)  # 需要更改
        self.bhz_var = Variable(torch.zeros(z_dim), requires_grad=True)

        #self.Wzh_tem = Variable(torch.nn.init.xavier_normal_(torch.empty(h_dim, z_dim)), requires_grad=True)
        #self.bzh_tem = Variable(torch.zeros(h_dim), requires_grad=True)
        
        self.Wzx = Variable(torch.nn.init.xavier_normal_(torch.empty(y_dim, z_dim)), requires_grad=True)  # 需要更改
        self.bzx = Variable(torch.zeros(y_dim), requires_grad=True)
        
        self.params = [self.Whz_mu, self.bhz_mu, self.Whz_var, self.bhz_var, self.Wzx, self.bzx]
        self.params.append(self.Wxh)
        self.params.append(self.bxh)
        #self.params.append(self.Wzh_tem)
        #self.params.append(self.bzh_tem)

    def forward(self, x):
        # See the autograd section for explanation of what happens here.
        self.z_mu, self.z_var = self.Q(x)
        z = self.sample_z(self.z_mu, self.z_var)
        y_pred = self.P(z)
        return y_pred
    
    # =============================== Q(z|X) ======================================
    def Q(self, x):
        h = F.relu(F.linear(x, self.Wxh, self.bxh))
        z_mu = F.linear(h, self.Whz_mu, self.bhz_mu)
        z_var = F.linear(h, self.Whz_var, self.bhz_var)
        return z_mu, z_var

    def sample_z(self, mu, log_var):
        eps = Variable(torch.randn(mu.shape[0], mu.shape[1]))
        return mu + torch.exp(log_var / 2) * eps

    # =============================== P(X|z) ======================================
    def P(self, z):
        #h = F.linear(z, self.Wzh_tem, self.bzh_tem)
        X = F.linear(z, self.Wzx, self.bzx)
        return X
    
# =============================== TRAINING ====================================  
model = vae(z_dim = Z_dim, x_dim = D_in, y_dim = D_out, h_dim = H_in)   
optimizer = torch.optim.Adam(model.params, lr=Learning_rate)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [ ]:
# mini-batch

start = time.time()
for epoch in range(1000):   # 训练所有!整套!数据
    for step, (batch_x, batch_y) in enumerate(loader):  # 每一步 loader 释放一小批数据用来学习
        # Forward
        y_pred = model(batch_x)

        # Loss
        recon_loss = F.mse_loss(y_pred, batch_y, size_average=False)
        kl_loss = torch.mean(0.5 * torch.sum(torch.exp(model.z_var) + model.z_mu**2 - 1. - model.z_var, 1))
        loss = recon_loss + kl_loss

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print and plot every now and then
    if epoch % 10 == 0:
        print('Iter-{}; Loss: {:.4}'.format(epoch, recon_loss.data[0]))
end = time.time()        
print('VAE cal_time is {}'.format(end-start))

y_pred_val = model(x_val)
recon_loss_val = F.mse_loss(y_pred_val, y_val, size_average=False)
print('VAE val_loss is {}'.format(recon_loss_val.item()))

Iter-0; Loss: 51.16
Iter-10; Loss: 3.57
Iter-20; Loss: 3.811
Iter-30; Loss: 1.582
Iter-40; Loss: 1.503
Iter-50; Loss: 1.655
Iter-60; Loss: 1.333
Iter-70; Loss: 2.648
Iter-80; Loss: 1.337


In [7]:
# Whole batch

start = time.time()
for t in range(20000):
    y_pred = model(x)
    
    recon_loss = F.mse_loss(y_pred, y, size_average=False)
    kl_loss = torch.mean(0.5 * torch.sum(torch.exp(model.z_var) + model.z_mu**2 - 1. - model.z_var, 1))
    loss = recon_loss + kl_loss
    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #scheduler.step(loss)
    
    # Print and plot every now and then
    if t % 1000 == 0:
        print('Iter-{}; Loss: {:.4}'.format(t, recon_loss.data[0]))
end = time.time()
print('VAE net cal_time is {}'.format(end-start))
y_pred_val = model(x_val)
recon_loss_val = F.mse_loss(y_pred_val, y_val, size_average=False)
print('VAE val_loss is {}'.format(recon_loss_val.item()))

Iter-0; Loss: 3.115
Iter-1000; Loss: 2.977
Iter-2000; Loss: 3.125
Iter-3000; Loss: 2.98
Iter-4000; Loss: 2.927
Iter-5000; Loss: 2.945
Iter-6000; Loss: 2.8
Iter-7000; Loss: 2.866
Iter-8000; Loss: 2.994
Iter-9000; Loss: 2.916
Iter-10000; Loss: 2.948
Iter-11000; Loss: 3.086
Iter-12000; Loss: 2.898
Iter-13000; Loss: 2.908
Iter-14000; Loss: 2.763
Iter-15000; Loss: 2.933
Iter-16000; Loss: 2.813
Iter-17000; Loss: 2.787
Iter-18000; Loss: 2.873
Iter-19000; Loss: 2.902
VAE net cal_time is 17.38748788833618
VAE val_loss is 1.361611247062683


In [ ]:
optimizer = torch.optim.Adam(model.params, lr=1e-4)

In [12]:
pred_show = y_pred_val[:1].clone().detach().numpy()
val_show = y_val[:1].clone().detach().numpy()
pred_show.reshape((-1, 3))
val_show.reshape((-1, 3))
print(pred_show)
print(val_show)

[[ 0.72489643  0.58723855  0.29059172  0.72132415  0.5884636   0.29146591
   0.71748704  0.58874875  0.29252183  0.71353656  0.58920979  0.29329067
   0.70901805  0.58949703  0.29402959]]
[[ 0.72544062  0.58681333  0.29145133  0.72657627  0.58678436  0.29277164
   0.72398645  0.5860998   0.29240772  0.72330785  0.58590436  0.29178232
   0.72071803  0.5858345   0.29009813]]


In [ ]:
optimizer = torch.optim.Adam(model.params, lr=1e-3)

In [ ]:
y_display = y_pred_val.detach().numpy()
y_display = y_display.astype('float').reshape(-1, 3)
y_display = np.multiply(y_display, (maxdata - mindata)) + mindata
#print(y_display)

y_display_val = y_val.detach().numpy()
y_display_val = y_display_val.astype('float').reshape(-1, 3)
y_display_val = np.multiply(y_display_val, (maxdata - mindata)) + mindata
#print(y_display_val)

#data = pd.DataFrame(y_pred_val.detach().numpy())
data = pd.DataFrame(y_display)
data.to_csv("./model_output/y_display.csv",index=False,header=False)
data = pd.DataFrame(y_display_val)
data.to_csv("./model_output/y_display_val.csv",index=False,header=False)

In [ ]:
# 仅保存和加载模型参数(推荐使用)
print(model.state_dict())
torch.save(model.state_dict(), 'params.pkl')

model.load_state_dict(torch.load('params_prone.pkl'))

params = model.state_dict()
for k,_ in params.items():
    print(k)

In [ ]:
w1 = params['0.weight'].detach().numpy()
print(w1.shape)
data = pd.DataFrame(w1)
data.to_csv("./data/w1.csv",index=False,header=False)

b1 = params['0.bias'].detach().numpy()
print(b1.shape)
data = pd.DataFrame(b1)
data.to_csv("./data/b1.csv",index=False,header=False)

w2 = params['2.weight'].detach().numpy()
print(w2.shape)
data = pd.DataFrame(w2)
data.to_csv("./data/w2.csv",index=False,header=False)

b2 = params['2.bias'].detach().numpy()
print(b2.shape)
data = pd.DataFrame(b2)
data.to_csv("./data/b2.csv",index=False,header=False)